### 김민섭님 코드

In [2]:
from saida_gym.starcraft.avoidObservers import AvoidObservers
import numpy as np
import math
import tensorflow as tf

In [4]:
env = AvoidObservers(frames_per_step=4, action_type=0, move_angle=30, move_dist=5, verbose=0, \
                     local_speed=0, no_gui=False, auto_kill=True, \
                     bot_runner=r"..\SAIDA_RL\cpp\Release\SAIDA\SAIDA.exe")

Initialize...
Shared Memory create 
SAIDA_AO3636 Shared memory found.


In [7]:
def make_state(me, en):
    MAX_ENEMY = 10
    ob_list = []
    for m in me:
        ob_list.append(scale_pos(m.pos_x))
        ob_list.append(scale_pos(m.pos_y))
        ob_list.append(scale_velocity(m.velocity_x))
        ob_list.append(scale_velocity(m.velocity_y))
        ob_list.append(scale_angle(m.angle))
    count = 0
    for e in en:
        ob_list.append(scale_pos(e.pos_x))
        ob_list.append(scale_pos(e.pos_y))
        ob_list.append(scale_velocity(e.velocity_x))
        ob_list.append(scale_velocity(e.velocity_y))
        ob_list.append(scale_angle(e.angle))
        count += 1
        if count == MAX_ENEMY: break
    if count != MAX_ENEMY:
        for k in range(MAX_ENEMY-count):
            for j in range(5):
                ob_list.append(0.0)
    return np.array(ob_list)

In [10]:
observation = env.reset()

In [11]:
state = make_state(observation.my_unit, observation.en_unit)
state

array([ 6.4000000e+01,  1.2400000e+02,  0.0000000e+00,  0.0000000e+00,
        5.0000000e-01,  7.3000000e+01,  1.0900000e+02,  6.0937500e-01,
        1.6796875e-01, -9.1406250e-01,  4.9000000e+01,  1.1100000e+02,
       -6.1328125e-01,  1.6796875e-01, -8.5937500e-02,  4.5000000e+01,
        1.2200000e+02, -6.0937500e-01,  1.7968750e-01, -7.8125000e-03,
        5.8000000e+01,  1.0500000e+02,  6.2890625e-01,  4.6875000e-02,
       -9.7656250e-01,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00])

In [5]:
def scale_velocity(v):
    return v


def scale_angle(angle):
    return (angle - math.pi) / math.pi


def scale_pos(pos):
    return int(pos / 16)


def fx(x):
    return -x + 2.5


def reward_function(state, temp_reward, episode, action):
    """temp = np.arange(30)
    temp = state.reshape(-1,5)[:,[0,1]]
    min_l2 = 20
    for i in range(5):
        l2 = np.sqrt(np.sum((temp[0]-temp[i+1])**2))
        #print(l2)
        if min_l2 > l2:
            min_l2 = l2
    #print(temp[0])
    reward = fx(min_l2/2.0) - (temp[0,1]/25.0)
    if temp_reward == 5:
        reward += temp_reward * 2
    elif temp_reward == -5:
        reward += temp_reward"""
    if episode < 50:
        if action[0] >= 7:
            reward = 0.3
        else:
            reward = -0.3
        if temp_reward == 5 or temp_reward == -5:
            reward = temp_reward
    else:
        reward = 0.0
        if temp_reward == 5 or temp_reward == -5:
            reward = temp_reward
    return reward

In [12]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=[None, 55])
TD_error = tf.placeholder(dtype=tf.float32, shape=[None])
Action = tf.placeholder(dtype=tf.int32, shape=[None])
TD_target = tf.placeholder(dtype=tf.float32, shape=[None])

with tf.variable_scope('shared_variables'):
    W1 = tf.get_variable("W1", shape=[55, 256], initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([256]))
    out1 = tf.matmul(X, W1) + b1
    fc1 = tf.nn.relu(out1)

    W2 = tf.get_variable("W2", shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([256]))
    out2 = tf.matmul(fc1,W2) + b2
    fc2 = tf.nn.relu(out2)

with tf.variable_scope('policy_variables'):
    W3 = tf.get_variable("W3", shape=[256,12], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([12]))
    policy_output = tf.matmul(fc2,W3) + b3
    policy = tf.nn.softmax(policy_output, 1)
    log_policy = tf.log(tf.diag_part(tf.gather(tf.transpose(policy), Action)))

with tf.variable_scope('value_variables'):
    W31 = tf.get_variable("W31", shape=[256,1], initializer=tf.contrib.layers.xavier_initializer())
    b31 = tf.Variable(tf.random_normal([1]))
    value = tf.reshape(tf.matmul(fc2,W31) + b31, [-1])

loss = -tf.reduce_mean(log_policy*TD_error) + tf.reduce_mean((TD_target-value)**2)

SGD = tf.train.GradientDescentOptimizer(0.0001)
update = SGD.minimize(loss)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
GAMMA = 0.9
episode_reward = []
ACTION_SPACE = 12

for episode in range(10000):
    state_list, action_list, td_error_list, td_target_list = [], [], [], []
    reward_sum = 0.0
    observation = env.reset()
    state = make_state(observation.my_unit, observation.en_unit)
    state_list.append(state)
    p_s = sess.run(policy, feed_dict={X:state.reshape(1,-1)}).reshape(-1)
    action = [np.random.choice(np.arange(ACTION_SPACE), p=p_s)]
    action_list.append(action[0])
    while True:
        observation, temp_reward, done, _ = env.step(action)
        next_state = make_state(observation.my_unit, observation.en_unit)
        reward = reward_function(next_state, temp_reward, episode, action)
        reward_sum += reward
        next_V = sess.run(value, feed_dict={X:next_state.reshape(1,-1)})[0]
        V = sess.run(value, feed_dict={X:state.reshape(1,-1)})[0]
        td_error = reward + GAMMA*next_V - V
        td_target_list.append(reward + GAMMA*next_V)
        td_error_list.append(td_error)
        if done: break
        state = next_state
        state_list.append(state)
        p_s = sess.run(policy, feed_dict={X:state.reshape(1,-1)}).reshape(-1)
        action = [np.random.choice(np.arange(ACTION_SPACE), p=p_s)]
        action_list.append(action[0])
    sess.run(update, feed_dict={X:np.array(state_list), TD_error:np.array(td_error_list),\
                                TD_target:np.array(td_target_list), Action:np.array(action_list)})
    if episode % 100 == 99:
        print('episode : %d, reward_sum : %f'%(episode, reward_sum))
        episode_reward.append(reward_sum)

episode : 99, reward_sum : -5.000000
episode : 199, reward_sum : -5.000000


KeyboardInterrupt: 